In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from cleaning import *

%matplotlib inline

In [174]:
data = pd.read_pickle('../junyi_df.pickle')



In [175]:
data = drop_columns(data)


In [176]:
data = change_bool(data)


In [177]:
convert_dates(data, 'time_done')

In [178]:
#Choosing Date Range
data = select_dates(data, 'time_done', '1/1/2014', '3/31/2014')


In [179]:
data.head()

,user_id,topic_mode,suggested,review_mode,time_done,time_taken,correct,count_attempts,hint_used,count_hints,earned_proficiency,points_earned
5,135350,0,1,0,2014-02-19 08:46:32.220080,9,1,1,0,0,0,225
6,126493,0,0,0,2014-03-21 01:56:56.571340,19,0,2,0,0,0,0
8,91712,0,1,0,2014-03-26 04:41:24.278610,4,1,1,0,0,0,195
9,13719,0,0,0,2014-03-06 09:30:38.458960,2,1,1,0,0,0,5
43,41750,1,1,0,2014-03-11 16:35:37.248730,15,1,1,0,0,0,225


In [180]:
#Creating User Table
users = data[['user_id', 'count_hints']].groupby('user_id').sum()

In [181]:
user_table = pd.DataFrame(users)

In [182]:
user_table

,count_hints
user_id,
4,0
16,3
26,0
29,0
35,18
36,7
39,0
46,0
47,0


In [183]:
#Creating a More than 10 columns
total_usage = data['user_id'].value_counts().sort_index()

In [184]:
user_table['more_than_ten_activities'] = (total_usage > 10).astype(int)

In [185]:
user_table['more_than_100_activities'] = (total_usage > 100).astype(int)
user_table['more_than_1000_activities'] = (total_usage > 1000).astype(int)

In [186]:
#Creating a More than X day columns

In [187]:
active_from_last= (data['time_done'].max() - (user_time['time_done'].max())).dt.days
user_table['active_from_last'] = active_from_last

In [188]:
user_time = data.groupby('user_id')
rng = (user_time['time_done'].max() - user_time['time_done'].min()).dt.days

In [189]:
user_table['active_more_than_1_day'] = (rng > 0).astype(int)

In [190]:
user_table['active_at_least_5_days'] = (rng >= 4).astype(int)
user_table['active_at_least_10_days'] = (rng >= 9).astype(int)


In [191]:
#Counting User attempts
user_attempts = data[['user_id', 'count_attempts']].groupby('user_id').sum()
user_table['sum_attempts'] = user_attempts

In [192]:
# Creating Target
target_data = pd.read_pickle('../junyi_df.pickle')
convert_dates(target_data, 'time_done')
target_data = select_dates(target_data, 'time_done', '4/1/2014', '4/7/2014')




In [193]:
#sorted(list(set(target_data['user_id'])))

In [194]:
#list(user_table.index)

In [ ]:
target = dict()

for user in list(user_table.index):
    if user in set(target_data['user_id']):
            target[user] = 1
    else: 
            target[user] = 0

In [ ]:
target

In [ ]:
#user_table.drop(columns='target', inplace=True)

In [ ]:
user_table['target'] = target.values()

In [ ]:
user_table.head()

In [ ]:
#user_table.drop(columns=['active_more_than_1', 'active_at_least_5', 'active_at_least_10'], inplace=True)

In [ ]:
user_table.head()

In [ ]:
#Save Table 

In [169]:
#Save Table 
user_table.to_csv('../users_v2.csv')

In [ ]:
#Getting last ten 10

In [65]:
user_indexed = data.set_index('user_id').sort_values(by='time_done')

In [66]:
user_indexed.head()

,topic_mode,suggested,review_mode,time_done,time_taken,correct,count_attempts,hint_used,count_hints,earned_proficiency,points_earned
user_id,,,,,,,,,,,
237856,0,0,0,2014-01-01 00:00:03.148100,6,1,1,0,0,0,75
154395,0,0,1,2014-01-01 00:00:14.107860,22,1,1,0,0,0,5
119799,1,1,0,2014-01-01 00:00:15.110180,10,1,1,0,0,0,225
154395,0,0,1,2014-01-01 00:00:19.035330,4,1,1,0,0,0,14
119799,1,1,0,2014-01-01 00:00:24.901530,5,1,1,0,0,0,225


In [67]:
user_indexed.loc[35, :]

,topic_mode,suggested,review_mode,time_done,time_taken,correct,count_attempts,hint_used,count_hints,earned_proficiency,points_earned
user_id,,,,,,,,,,,
35,0,0,0,2014-01-08 06:43:24.504230,255,0,7,1,2,0,0
35,0,0,0,2014-01-09 09:01:02.373640,60,0,3,0,0,0,0
35,0,0,0,2014-01-09 09:02:21.835700,45,1,1,0,0,0,75
35,0,0,0,2014-01-09 09:02:40.267920,11,1,1,0,0,0,75
35,0,0,0,2014-01-09 09:02:50.194020,18,0,2,0,0,0,0
35,0,0,0,2014-01-09 09:03:05.104680,13,0,2,0,0,0,0
35,0,0,0,2014-01-09 09:03:24.794640,14,1,1,0,0,0,75
35,0,0,0,2014-01-09 09:03:34.756280,8,1,1,0,0,0,75
35,0,0,0,2014-01-09 09:03:46.724700,25,0,3,0,0,0,0


In [81]:


user_indexed.loc[35, 'count_attempts'][-10:].sum()

56

In [79]:

y = user_indexed.loc[35, 'count_hints'][-10:].sum()

In [80]:
y

8

In [ ]:
for user_id in set(new_data.index.values):
#     days_active[user_id] = get_active_weeks(user_id)

In [ ]:
def get_last_10_hints(user_id):
    try: 
        return len(set(new_data.loc[user_id, 'week']))
    except:
        return 0

In [ ]:
#Creating a More than 1 day co